# 64B/66B  Encoding: 64-bit blocks in to 66-bit block codes

## Objective
Using the [RD53A Integrated Circuit Manual](https://cds.cern.ch/record/2287593?ln=en), we like to design a data stream resembling information being sent out of the CMS Inner Tracker detector. Below is the excerpt from the manual, explaining the data output protocol.

<td> <img src="DataStreamFigs/Screen%20Shot%202020-04-21%20at%204.06.30%20PM.png" alt="Drawing"/> </td>

There are two types of frames making up the output format, data frames and register frames (each contain 66 bits). There is always a specified amount of N data frames (a through d) before the register frame (e) and the pattern repeats. The amount 'N:1' of data frames to register is user programable, depending on the what fraction of bandwidth is reserve for the hit data. The output is encoded on 1 to 4 parallel lanes, that are also programable. Making matters more complicated, there are 4 different types of data frames, 2 types for transmitting hit information (a,b) and 2 types for when there are no hits (c,d).

The interval N is used on every lane regardless of how many lanes are active. For example, with $N_D=48$, 2% of the output bandwidth is permanently unavailable for the hit data (in addition to the 3% consumed by the 2-bit header). Therefore, at $4\times1.28$ Gbps output bandwidth, this 2% is sufficient for the maximum possible register readback of 64 Mbps, since 2% of 5 Gbps = 100 Mbps.

Lets generate such a data frame. 

### Header (2 bits)

The first step is to initialize the possible data and register frames. Each frame has a 2-bit sync header that can be 01 or 10. Header 01 always indicates an Aurora data frame type, while the 10 header frames can be of user data type or Aurora command type.

In [1]:
from random import randint
import re
            
data1 = [0]*66   ## Aurora Data Frame 
data2 = [0]*66   ## User Data / Aurora Command Type
data3 = [0]*66   ## No Hits; Idle Framing Words 
data4 = [0]*66   ## No Hits; Aurora Framing Words 
reg   = [0]*66   ## Register Frame

# Assign 2-bit headers
data1[0:2] = [0,1]
data2[0:2] = [1,0]
data3[0:2] = [1,0]
data4[0:2] = [1,0]
reg[0:2]   = [1,0]
print('reg[0:2]:', reg[0:2], '\ndata_a[0:2]:', data1[0:2], '\ndata_b[0:2]:', data2[0:2], '\ndata_c[0:2]:', data3[0:2], '\ndata_d[0:2]:', data4[0:2])

reg[0:2]: [1, 0] 
data_a[0:2]: [0, 1] 
data_b[0:2]: [1, 0] 
data_c[0:2]: [1, 0] 
data_d[0:2]: [1, 0]


## Register

### Aurora Codes (8 bits)
After adding the header bits, the next 8 bits are one of the five possible Aurora codes in the periodic register frame, as refered to 'zz' in the first picture.

<td> <img src="DataStreamFigs/Screen%20Shot%202020-04-21%20at%205.28.18%20PM.png" alt="Drawing"/> </td>

In [2]:
def getHexToBinaryData(data_loc, bit_size):
    x_bit_loc = "{0:08b}".format(int(data_loc, 16)).replace("0b","").zfill(bit_size)# remove "0b" and pad with 0s if needed 
    x_bits = [int(i) for i in x_bit_loc]  ## convert elements from type string to type int
    return x_bits

def get_aurora_code(zz):
    hex_choice = ''
    if   zz==0: hex_choice = '0xB4'  
    elif zz==1: hex_choice = '0x55'
    elif zz==2: hex_choice = '0x99'
    elif zz==3: hex_choice = '0xD2'
    elif zz==4: hex_choice = '0xCC'   
    aurora_code = getHexToBinaryData(hex_choice, 8)
    return aurora_code

reg[2:10] = get_aurora_code(randint(0,4))

print('reg[2:10] = ', reg[2:10]) 

reg[2:10] =  [1, 1, 0, 0, 1, 1, 0, 0]


After the 8-bit code, followed by the sync header, there are 56 bits available for user information, which are allocated as two registers (10-bit extended address plus 16-bit value = 26
bits) plus 4 status bits:

__2x( [e-address (10 bits)] [value (16 bits)] ) [status (4 bits)]__

### Status Codes (4 bits)

Below is the defination of $(2^4=16)$ possible 4-bit status codes. Lets not use the spare bits, meaning restraining to only 7 decimal choices.

<td> <img src="DataStreamFigs/Screen%20Shot%202020-04-21%20at%206.28.35%20PM.png" alt="Drawing"/> </td>

In [3]:
def getDecimalToBinaryData(data_loc, bit_size):
    x_bit_loc = str(bin(data_loc)).replace("0b","").zfill(bit_size) # remove "0b" and pad with 0s if needed
    x_bits = [int(i) for i in x_bit_loc] 
    return x_bits

status_code = randint(0,6) # (0,15) in case user defines additional status for (7,15) bits              
reg[10:14] = getDecimalToBinaryData(status_code, 4)
print('decimal: ',status_code,'==> reg[10:14]',reg[10:14]) 

decimal:  0 ==> reg[10:14] [0, 0, 0, 0]


### Auto-fill A/B register address values (16 bits)
The two 16-bit registers are denoted $A_i$ and $B_i$, where $i$ is the lane number (0 to 3). The automatic filling of the $A_i$ and $B_i$ registers is controlled by eight configuration registers Auto-$A_i$ and Auto-$B_i$, which have default values, but which the user is free to change. 

<td> <img src="DataStreamFigs/Screen%20Shot%202020-04-21%20at%209.19.13%20PM.png" alt="Drawing"/> </td>

The auto-fill register addresses are specific to each lane. Thus if only lane 0 is used then only Auto-$A_0$ and Auto-$B_0$ are functional. 

In [4]:
reg[24:32] = getDecimalToBinaryData(136, 8)
reg[32:40] = getDecimalToBinaryData(136, 8)
reg[50:58] = getDecimalToBinaryData(130, 8)
reg[58:66] = getDecimalToBinaryData(130, 8)

print('reg[24:40] = ', reg[24:40], '\nreg[50:66] = ', reg[50:66],'\n')

reg[24:40] =  [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0] 
reg[50:66] =  [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0] 



### Extended register address (10 bits)
The extended 10 bit address can have two of the following cases 
1. MSB=0, followed by the 9-bit global register address (0-511) 
2. MSB=1, followed by the 9-bit offset pixel row address (512-895)

The separation of the output into two time multiplexed channels guarantees a certain bandwidth for both data and register information without the need for a complex priority arbitration containing safeguards against all possible pathologies.

In [5]:
extended_address = randint(0,895)
reg[14:24]  = getDecimalToBinaryData(extended_address, 10)
reg[40:50]  = getDecimalToBinaryData(extended_address, 10)
print('extended address:', extended_address)
print('reg[14:24]:', reg[14:24],'\nreg[40:50]:', reg[40:50]) 

extended address: 487
reg[14:24]: [0, 1, 1, 1, 1, 0, 0, 1, 1, 1] 
reg[40:50]: [0, 1, 1, 1, 1, 0, 0, 1, 1, 1]


Lets break it down and print the full register.

In [6]:
print('reg[0:2] = ', reg[0:2], '\nreg[2:10] = ', reg[2:10], '\nreg[10:14] = ', reg[10:14], 
      '\nreg[14:24] = ', reg[14:24],'\nreg[24:40] = ', reg[24:40], '\nreg[40:50] = ', reg[40:50],
      '\nreg[50:66] = ', reg[50:66],'\n')
print ('register size: ', len(reg), '\nreg = ', reg)
#for c, i in enumerate(reg):
#    print (c, i)
#    c = c+1

reg[0:2] =  [1, 0] 
reg[2:10] =  [1, 1, 0, 0, 1, 1, 0, 0] 
reg[10:14] =  [0, 0, 0, 0] 
reg[14:24] =  [0, 1, 1, 1, 1, 0, 0, 1, 1, 1] 
reg[24:40] =  [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0] 
reg[40:50] =  [0, 1, 1, 1, 1, 0, 0, 1, 1, 1] 
reg[50:66] =  [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0] 

register size:  66 
reg =  [1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]


## Idle Frames: Data(c) and Data(d)

When there are no hits, two possible frames may be sent. Both start with a binary “10” header. Followed by the header, frame Data(c) has a 0x1E hex or 8-bit code 00011110, with the remaining 48 bits set to 0. The other idle data frame, Data(d), followed by the header has a 0x78 hex or 8-bit code 01111000, with another 8-bit code from the Aurora code zz table. Then, there are 48-bits of “unused” data, which could mean that we do not read that data.  We will keep them to set to 0 as we did in Data(c) through initialization.

In [7]:
data3[2:10]  = getHexToBinaryData('0x1E', 8)

data4[2:10]  = getHexToBinaryData('0x78', 8)
data4[10:18] = get_aurora_code(randint(0,4)) ## Additional possible 8-bit Aurora code

print ('data_c: ', data3, '\ndata_d: ', data4)

data_c:  [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
data_d:  [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Hit or Event Header Frames: Data(a) and Data(b)

### Pixel address in Hit Data (32 bits)

The pixel addressing is hierarchical, first in cores (like postal codes), then regions within a core
(like the street address), and then a pixel pair forming 1-bit (like a town house in each street). Each 32-bit data register contains information from two 16-bit pixel quadcore addresses. In the readout the basic unit is the quad region, and thus in each data record the 16-bit pixel quad core address is given by a 

6 bit core column [15:10]=Core\_Col, 6 bit core row [9:4]=Core\_Row, and 3 bit core region [3:0]=Region\_in\_Core. 

This structure is shown graphically below. The address is followed by four 4-bit 'Time over Threshold' values, where the first value is the left-most pixel (left as defined in Fig below). __The 4 pixels are always in the same chip row with increasing column numbers. Thus the address of each group is given by a 4-column and a row.__

<td> <img src="DataStreamFigs/Screen%20Shot%202020-04-22%20at%201.51.05%20AM.png" alt="Drawing"/> </td>

For configuration the Core\_Col and Core\_Row values are preserved in registers 1 and 2, respectively, but the four Region\_in\_Core are divided between the two registers as follows. Additionally, there is a Pair\_in\_Region bit need (17 total bits instead of 16), because configuration is written in pixel pairs rather than quads.

Register 1 = [7:2]=Core\_Col, [1]=Region\_in\_Core,   [0]=Pair\_in\_Region

Register 2 = [8:3]=Core\_Row, [2:0]=Region\_in\_Core, [3:1]=Pair\_in\_Region

In this way, Register 1 has the traditional column pair meaning, while Register 2 has the row meaning.

### Data(b)

Data(b) frame used in transmitting data can either be of user data type or Aurora command type. From the first figure at the start, lets fill the first 32 bits. Followed by the 2 bit header (10), the data frame has a 0x1E hex or 8-bit 00011110 code, then another 0x04 hex or 8-bit 00000100, another 0x0000 hex or 16 bit 0000000000000000 code. After these codes, 32-bits of data are transmitted in the pixel address format explained above.

In [8]:
data2[2:10]   = getHexToBinaryData('0x1E', 8)  
data2[10:18]  = getHexToBinaryData('0x04', 8)
data2[18:34]  = getHexToBinaryData('0x0000', 16)

print('data2[2:10] = ', data2[2:10], '\ndata2[10:18] = ', data2[10:18], 
      '\ndata2[18:34] = ', data2[18:34])

data2[2:10] =  [0, 0, 0, 1, 1, 1, 1, 0] 
data2[10:18] =  [0, 0, 0, 0, 0, 1, 0, 0] 
data2[18:34] =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
col = randint(0,63)                   # highest number count in decimal for a 6 bit address
row = randint(0,63)                   # highest number count in decimal for a 6 bit address
region = randint(0,7)                 # highest number count in decimal for a 3 bit address

## Createss 2 pixel addresses for the data2 frame, that are displayed by increasing column number
if   col< 63: col2=col+1              
elif col==63: col2=col-1

## Create a 6 bit pixel column
data2[34:40] = getDecimalToBinaryData(col, 6)

## Create a second 6 bit pixel column
data2[50:56] = getDecimalToBinaryData(col2, 6)

## Creates a 6 bit pixel_row 
data2[41:47] = getDecimalToBinaryData(row, 6)
data2[57:63] = getDecimalToBinaryData(row, 6)

## Creates a 3 bit region
data2[47:50]= getDecimalToBinaryData(region, 3)
data2[63:66]= getDecimalToBinaryData(region, 3)

### -----> Why bits 40 and 56 are left with value 0?
print('data2[34:40] = ', data2[34:40], ', data2[40:41] = ', data2[40:41],
      '\ndata2[41:47] = ', data2[41:47], 
      '\ndata2[47:50] = ', data2[47:50],
      '\ndata2[50:56] = ', data2[50:56], ', data2[56:57] = ',  data2[56:57],
      '\ndata2[57:63] = ', data2[57:63],
      '\ndata2[63:66] = ', data2[63:66]
     ) 
print ('\ndata_b = ', data2)
#for c, i in enumerate(data2):
#    print (c, i)
#    c = c+1

data2[34:40] =  [0, 0, 1, 0, 0, 0] , data2[40:41] =  [0] 
data2[41:47] =  [0, 1, 1, 0, 1, 0] 
data2[47:50] =  [0, 1, 1] 
data2[50:56] =  [0, 0, 1, 0, 0, 1] , data2[56:57] =  [0] 
data2[57:63] =  [0, 1, 1, 0, 1, 0] 
data2[63:66] =  [0, 1, 1]

data_b =  [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1]


### Data(a)
The Aurora data frame begins with a 2-bit “01” header. Then are the two 32-bit registers corresponding to Hit data or Event Header data.

In [10]:
col = randint(0,63)                  # highest number count in decimal for a 6 bit address
row = randint(0,63)                  # highest number count in decimal for a 6 bit address
region = randint(0,7)                # highest number count in decimal for a 3 bit address

## Createss 4 pixel addresses for the data1 frame, that are displayed by increasing column number
if col < 60:
    col2=col+1
    col3=col+2
    col4=col+3
elif col>60:
    col2=col-1
    col3=col-2
    col4=col-3
    
## Create four 6 bit pixel_columns 
data1[2:8]  = getDecimalToBinaryData(col, 6)
data1[18:24]= getDecimalToBinaryData(col2, 6)
data1[34:40]= getDecimalToBinaryData(col3, 6)
data1[50:56]= getDecimalToBinaryData(col4, 6)

## Create four 6 bit row_columns
data1[9:15] = getDecimalToBinaryData(row, 6)
data1[25:31]= getDecimalToBinaryData(row, 6)
data1[41:47]= getDecimalToBinaryData(row, 6)
data1[57:63]= getDecimalToBinaryData(row, 6)

## Creates possible 3 bit regions 
data1[15:18]= getDecimalToBinaryData(region, 3)
data1[31:34]= getDecimalToBinaryData(region, 3) 
data1[47:50]= getDecimalToBinaryData(region, 3) 
data1[63:66]= getDecimalToBinaryData(region, 3) 

### -----> Why bits 8, 24, 40 and 56 are left with value 0?
print('data1[2:8] = ',data1[2:8], ', data1[8:9] = ',data1[8:9],
      '\ndata1[9:15] = ',data1[9:15], 
      '\ndata1[15:18] = ',data1[15:18], 
      '\ndata1[18:24] = ',data1[18:24], ', data1[24:25] = ',data1[24:25],
      '\ndata1[25:31] = ',data1[25:31], 
      '\ndata1[31:34] = ',data1[31:34], 
      '\ndata1[34:40] = ',data1[34:40], ', data1[40:41] = ',data1[40:41],
      '\ndata1[41:47] = ',data1[41:47],
      '\ndata1[47:50] = ',data1[47:50],
      '\ndata1[50:56] = ',data1[50:56], ', data1[56:57] = ',data1[56:57],
      '\ndata1[57:63] = ',data1[57:63],
      '\ndata1[63:66] = ',data1[63:66],
     )

print ('\ndata_a = ', data1)
#for c, i in enumerate(data1):
#    print (c, i)
#    c = c+1

data1[2:8] =  [1, 0, 0, 1, 0, 0] , data1[8:9] =  [0] 
data1[9:15] =  [1, 1, 0, 1, 1, 1] 
data1[15:18] =  [1, 1, 1] 
data1[18:24] =  [1, 0, 0, 1, 0, 1] , data1[24:25] =  [0] 
data1[25:31] =  [1, 1, 0, 1, 1, 1] 
data1[31:34] =  [1, 1, 1] 
data1[34:40] =  [1, 0, 0, 1, 1, 0] , data1[40:41] =  [0] 
data1[41:47] =  [1, 1, 0, 1, 1, 1] 
data1[47:50] =  [1, 1, 1] 
data1[50:56] =  [1, 0, 0, 1, 1, 1] , data1[56:57] =  [0] 
data1[57:63] =  [1, 1, 0, 1, 1, 1] 
data1[63:66] =  [1, 1, 1]

data_a =  [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]


# Bit stream

With one register and four data frames, each of 66 bit, let write a bit stream. For a bit size of 32768, we need to write 32768/66 ~ 492 frames. If we choose to have data frames to appear N=4 followed by register, then we must have a counter of size 496 to create the pattern for example: R + D4+D4+D4+D4 + R + D1+D1+D1+D1 + R + ..... We must pad the final bitstream to zeros to ensure the total size of bits is 32768.

In [13]:
'''
print('reg: ', reg, '\n')
print('\ndata1:', data1, '\n')
print('\ndata2:', data2, '\n')
print('\ndata3:', data3, '\n')
print('\ndata4:', data4, '\n')
'''
def padZeros(bit_stream, size):
    a = ''.join(str(bit) for bit in bit_stream)
    b = a.zfill(size)
    final_stream = [int(i) for i in b] 
    return final_stream

f=open("DataStream.coe","w+")                     ## Sets up the coe file
f.write("*********************************************************\n")
f.write("*********** Dual Port Block Memory .COE file ************\n")
f.write("*********************************************************\n")
f.write("; memory initialization file for Dual Port Block Memory,\n")
f.write("; v3.0 or later.\n")
f.write(";\n")
f.write("; This .COE file specifies the contents for a block memory\n")
f.write("; of depth=1048576, and width=32. In this case, values are \n")
f.write("; specified in hex format.\n")
f.write("memory_initialization_radix=16\n")
f.write("memory_initialization_vector=\n")

bits = 1048576     ## 32768 * 32, here 32768 is the size of ROM and 32 bits is the width
bit_stream = []

## Frames to call 32768/66~15887.5. As we must start with the register, 
## lets keep the counter to 15884 for a bit stream with N=4 data frames
## Then pad it to zeros to get the right size of bits (1048576).

count = 0; frame_choice=0;
while count < 15884:                                       
    if ((count%4==0) & (count!=0)):  ## Makes the register frame periodically appear after every N = 4 data frames
        bit_stream.extend(reg)
        frame_choice=randint(1,4)
    elif(count%4!=0):               ## Randomly choose one of the four possible data frames and repeat it 4 times    
        if   frame_choice==1: bit_stream.extend(data1)
        elif frame_choice==2: bit_stream.extend(data2)
        elif frame_choice==3: bit_stream.extend(data3)
        elif frame_choice==4: bit_stream.extend(data4)          
    count = count+1 
     
if len(bit_stream) < bits:  
    print('bitstream size', len(bit_stream), ', lets do padding by zeros')
    stream_to_write = padZeros(bit_stream, bits)
else:
    print('bitstream size', len(bit_stream), ', no zero padding required')
    stream_to_write = bit_stream
    
i = 0; b = '';
for j in range(32, bits+32, 32):  #(4, bits+4, 32)              
    #print (j)    
    a = ' '.join(str(bit) for bit  in stream_to_write[i:j] )
    a = re.sub(r"\s+", "", a)                # remove the white spaces
    a = "{0:0>8X}".format(int(a, 2))         # convert into hex, comment this line if you like to write in binary
    if j !=0 : b = a+','
    i = j                              
    if j == bits: b = b.replace(',', ';')    # add the semicolon in the last row
    print (b)    
    f.write(b+'\n')

#print(','.join(str(bit) for bit in stream_to_write))
#f.write(','.join(str(bit) for bit in stream_to_write)) 

stream_to_write.clear()
bit_stream.clear()
f.close()

bitstream size 1048080 , lets do padding by zeros
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
00000000,
0000B301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,
79E0A0A1,
E0400002,
0D324D38,
78100000,
834C934E,
1E040000,
20D324D3,
B301E788,
8879E0A0,
A1E00000,
00000000,
08780000,
00000000,
021E0000,
00000000,
00B301E7,
888879E0,
A0A1E040,
00020D32,
4D387810,
0000834C,
934E1E04,
000020D3,
24D3B301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A0A7,
89900000,
00000009,
E2640000,


01E78888,
79E0A0A1,
E0400002,
0D324D38,
78100000,
834C934E,
1E040000,
20D324D3,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,
79E0A0A1,
E0000000,
00000008,
78000000,
00000002,
1E000000,
00000000,
B301E788,
8879E0A0,
991BF95B,
F99BF9DB,
F646FE56,
FE66FE76,
FD91BF95,
BF99BF9D,
BFB301E7,
888879E0,
A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0A1E0,
00000000,
00000878,
00000000,
0000021E,
00000000,
000000B3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A1E04000,
020D324D,
38781000,
00834C93,
4E1E0400,
0020D324,
D3B301E7,
888879E0,
A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,


E7888879,
E0A0A1E0,
00000000,
00000878,
00000000,
0000021E,
00000000,
000000B3,
01E78888,
79E0A0A7,
89900000,
00000009,
E2640000,
00000002,
78990000,
00000000,
B301E788,
8879E0A0,
A1E04000,
020D324D,
38781000,
00834C93,
4E1E0400,
0020D324,
D3B301E7,
888879E0,
A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0A1E0,
00000000,
00000878,
00000000,
0000021E,
00000000,
000000B3,
01E78888,
79E0A0A1,
E0400002,
0D324D38,
78100000,
834C934E,
1E040000,
20D324D3,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,
79E0A0A1,
E0000000,
00000008,
78000000,
00000002,
1E000000,
00000000,
B301E788,
8879E0A0,
A1E04000,
020D324D,
38781000,
00834C93,
4E1E0400,
0020D324,
D3B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,


E0A0A1E0,
00000000,
00000878,
00000000,
0000021E,
00000000,
000000B3,
01E78888,
79E0A0A1,
E0400002,
0D324D38,
78100000,
834C934E,
1E040000,
20D324D3,
B301E788,
8879E0A0,
991BF95B,
F99BF9DB,
F646FE56,
FE66FE76,
FD91BF95,
BF99BF9D,
BFB301E7,
888879E0,
A0A78990,
00000000,
0009E264,
00000000,
00027899,
00000000,
0000B301,
E7888879,
E0A0A1E0,
00000000,
00000878,
00000000,
0000021E,
00000000,
000000B3,
01E78888,
79E0A0A1,
E0000000,
00000008,
78000000,
00000002,
1E000000,
00000000,
B301E788,
8879E0A0,
991BF95B,
F99BF9DB,
F646FE56,
FE66FE76,
FD91BF95,
BF99BF9D,
BFB301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A1E0,
4000020D,
324D3878,
10000083,
4C934E1E,
04000020,
D324D3B3,
01E78888,
79E0A0A7,
89900000,
00000009,
E2640000,
00000002,
78990000,
00000000,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0A78990,
00000000,
0009E264,
00000000,
00027899,
00000000,
0000B301,
E7888879,
E0A0991B,


00000000,
00B301E7,
888879E0,
A0A1E040,
00020D32,
4D387810,
0000834C,
934E1E04,
000020D3,
24D3B301,
E7888879,
E0A0A1E0,
4000020D,
324D3878,
10000083,
4C934E1E,
04000020,
D324D3B3,
01E78888,
79E0A0A1,
E0000000,
00000008,
78000000,
00000002,
1E000000,
00000000,
B301E788,
8879E0A0,
A1E00000,
00000000,
08780000,
00000000,
021E0000,
00000000,
00B301E7,
888879E0,
A0A1E040,
00020D32,
4D387810,
0000834C,
934E1E04,
000020D3,
24D3B301,
E7888879,
E0A0A1E0,
00000000,
00000878,
00000000,
0000021E,
00000000,
000000B3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,
79E0A0A1,
E0000000,
00000008,
78000000,
00000002,
1E000000,
00000000,
B301E788,
8879E0A0,
991BF95B,
F99BF9DB,
F646FE56,
FE66FE76,
FD91BF95,
BF99BF9D,


...

00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0A78990,
00000000,
0009E264,
00000000,
00027899,
00000000,
0000B301,
E7888879,
E0A0A1E0,
4000020D,
324D3878,
10000083,
4C934E1E,
04000020,
D324D3B3,
01E78888,
79E0A0A1,
E0000000,
00000008,
78000000,
00000002,
1E000000,
00000000,
B301E788,
8879E0A0,
A1E00000,
00000000,
08780000,
00000000,
021E0000,
00000000,
00B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0A1E040,
00020D32,
4D387810,
0000834C,
934E1E04,
000020D3,
24D3B301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,
79E0A0A1,
E0400002,
0D324D38,
78100000,
834C934E,
1E040000,
20D324D3,
B301E788,
8879E0A0,
A1E00000,
00000000,


A1E00000,
00000000,
08780000,
00000000,
021E0000,
00000000,
00B301E7,
888879E0,
A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A1E00000,
00000000,
08780000,
00000000,
021E0000,
00000000,
00B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A1E0,
00000000,
00000878,
00000000,
0000021E,
00000000,
000000B3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A1E04000,
020D324D,
38781000,
00834C93,
4E1E0400,
0020D324,
D3B301E7,
888879E0,
A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0A1E0,
4000020D,
324D3878,
10000083,
4C934E1E,
04000020,
D324D3B3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
991BF95B,


020D324D,
38781000,
00834C93,
4E1E0400,
0020D324,
D3B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A789,
90000000,
000009E2,
64000000,
00000278,
99000000,
000000B3,
01E78888,
79E0A0A7,
89900000,
00000009,
E2640000,
00000002,
78990000,
00000000,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A1E00000,
00000000,
08780000,
00000000,
021E0000,
00000000,
00B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A1E0,
4000020D,
324D3878,
10000083,
4C934E1E,
04000020,
D324D3B3,
01E78888,
79E0A0A7,
89900000,
00000009,
E2640000,
00000002,
78990000,
00000000,
B301E788,
8879E0A0,
A7899000,
00000000,


A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A1E00000,
00000000,
08780000,
00000000,
021E0000,
00000000,
00B301E7,
888879E0,
A0A1E000,
00000000,
00087800,
00000000,
00021E00,
00000000,
0000B301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A0A1,
E0000000,
00000008,
78000000,
00000002,
1E000000,
00000000,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0A78990,
00000000,
0009E264,
00000000,
00027899,
00000000,
0000B301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A0A7,
89900000,
00000009,
E2640000,
00000002,
78990000,
00000000,
B301E788,
8879E0A0,
A7899000,
00000000,
09E26400,
00000000,
02789900,
00000000,
00B301E7,
888879E0,
A0A78990,


F99BF9DB,
F646FE56,
FE66FE76,
FD91BF95,
BF99BF9D,
BFB301E7,
888879E0,
A0A1E040,
00020D32,
4D387810,
0000834C,
934E1E04,
000020D3,
24D3B301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A0A1,
E0400002,
0D324D38,
78100000,
834C934E,
1E040000,
20D324D3,
B301E788,
8879E0A0,
991BF95B,
F99BF9DB,
F646FE56,
FE66FE76,
FD91BF95,
BF99BF9D,
BFB301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0991B,
F95BF99B,
F9DBF646,
FE56FE66,
FE76FD91,
BF95BF99,
BF9DBFB3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A1E04000,
020D324D,
38781000,
00834C93,
4E1E0400,
0020D324,
D3B301E7,
888879E0,
A0991BF9,
5BF99BF9,
DBF646FE,
56FE66FE,
76FD91BF,
95BF99BF,
9DBFB301,
E7888879,
E0A0A1E0,
4000020D,
324D3878,
10000083,
4C934E1E,
04000020,
D324D3B3,
01E78888,
79E0A099,
1BF95BF9,
9BF9DBF6,
46FE56FE,
66FE76FD,
91BF95BF,
99BF9DBF,
B301E788,
8879E0A0,
A1E00000,
00000000,
